# Infeasibility UQ

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the QoI as the value of constraints, i.e. the infeasibility.

Dask is used to parallelise the evaluations for a SLURM cluster.


In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import shutil
import os

## Campaign to capture feasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals.

To start with, make just 4 inputs uncertain.


In [2]:
# Init cluster (describes a single node, or less if need less than that per worker)
# cluster = SLURMCluster(
#     cores=56,
#     processes=4,  # check docs
#     memory="192GB",
#     account="UKAEA-AP002-CPU",
#     walltime="01:00:00",
#     queue="cclake",
# )

# Need less than a full node per worker
cluster = SLURMCluster(
    cores=1,
    processes=1,
    memory="4GB",
    account="UKAEA-AP002-CPU",
    walltime="15:00:00",
    queue="cclake",
)
# cluster.scale(jobs=4)
cluster.adapt(minimum_jobs=10,maximum_jobs=128)  # 4 workers
# print(cluster.job_script())

# Connect Dask client to remote cluster
client = Client(cluster)
# Code from now on submitted to batch queue

# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_cluster", work_dir=WORK_DIR)


# Define parameter space
# The ranges here are taken from demo_confopt_1, the first optimisation in the previous study.
params = {
    "coreradius": {
        "type": "float",
        "min": 0.6,
        "max": 0.9,
        "default": 0.72
    },
    "ralpne": {
        "type": "float",
        "min": 0.072,
        "max": 0.0761,
        "default": 0.07268
    }, 
    "psepbqarmax": {
        "type": "float",
        "min": 7.359,
        "max": 11.04,
        "default": 10.71
    },
    "tbrnmn": {
        "type": "float",
        "min": 6954.0,
        "max": 12722.0,
        "default": 11730.88 
    },
    "etaech": {
        "type": "float",
        "min": 0.32,
        "max": 0.48,
        "default":0.41
    }, 
    "pinjalw": {
        "type": "float",
        "min": 40.0,
        "max": 60.0,
        "default": 47.78
    }, 
    "triang": {
        "type": "float",
        "min": 0.4,
        "max": 0.6,
        "default":0.56
    }, 
    "alstroh": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default":660600000.0 
    }, 
    "sig_tf_case_max": {
        "type": "float",
        "min": 464000000.,
        "max": 696000000.0,
        "default": 638600000.0
    }, 
    "walalw": {
        "type": "float",
        "min": 6.4,
        "max": 9.6,
        "default": 8.91
    }, 
    "sig_tf_wp_max": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default": 723600000.0
    }, 
    "aspect": {
        "type": "float",
        "min": 2.78,
        "max": 2.94,
        "default": 2.89
    }, 
    "etath": {
        "type": "float",
        "min": 0.35,
        "max": 0.405,
        "default": 0.40
    }, 
    "vdalw": {
        "type": "float",
        "min": 7.00,
        "max": 13.00,
        "default": 9.99
    },
    "tftmp": {
        "type": "float",
        "min": 4.00,
        "max": 6.00,
        "default": 4.750
    },
    "boundu(52)": {
        "type": "float",
        "min": 8.0,
        "max": 12.00,
        "default": 10.00
    },
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
}

# QoIs
# A list of iteration variables, ifail and sqsumsq.
qois = [
        "ifail",
        "sqsumsq",
        "bt",
        "te",
        "beta",
        "dene",
        "tfcth",
        "wallmw",
        "ohcth",
        "bigq",
        "bore",
        "betalim",
        "coheof",
        "cohbop",
        "kappa",
        "fvsbrnni",
        "itvar019",
        "itvar020",
        "jwptf",
        "vtfskv",
        "vdalw",
        "tdmptf",
        "thkcas",
        "thwcndut",
        "fcutfsu",
        "cpttf",
        "plhthresh",
        "tmargtf",
        "tmargoh",
        "oh_steel_frac",
        "pdivt",
        "powfmw",
        "rmajor",
        "tftmp",
        "fimp(13)",
        "ncycle"
]

# Create encoder and decoder
encoder1 = uq.encoders.GenericEncoder(
    template_fname="clean_demo_2023_IN.template", target_filename="IN.DAT"
)
encoder2 = uq.encoders.GenericEncoder(
    template_fname="MFILE.DAT.json", target_filename="MFILE.DAT.json"
)
decoder = uq.decoders.JSONDecoder(target_filename="MFILE.DAT.json", output_columns=qois)
my_multiencoder = uq.encoders.MultiEncoder(encoder1, encoder2)


cmd = "process -i IN.DAT --mfilejson --mfile MFILE.DAT"
actions = uq.actions.local_execute(my_multiencoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

# Create PCE sampler, 4 uncertainties
# vary = {
#     "aspect": cp.Uniform(2.785,2.935),
#     "triang": cp.Uniform(0.4, 0.6),
#     "psepbqarmax": cp.Uniform(8.7, 9.7),
#     "hfact": cp.Uniform(1.0, 1.2),
# }
vary={"coreradius": cp.Uniform(0.6,0.75),
      "ralpne": cp.Uniform(0.072,0.0761),
      "psepbqarmax": cp.Uniform(9.20,11.02),
      "tbrnmn": cp.Uniform(10782.05,12680.82),
      "etaech": cp.Uniform(0.40,0.48),
      "pinjalw": cp.Uniform(40.0,49.0),
      "triang": cp.Uniform(0.53,0.6),
      "alstroh": cp.Uniform(529700000.0,788900000.0),
      "sig_tf_case_max": cp.Uniform(580100000.0,695500000.0),
      "walalw": cp.Uniform(6.4,9.6),
      "sig_tf_wp_max": cp.Uniform(658600000.0,789200000.0),
      "aspect": cp.Uniform(2.89,2.90),
      "etath": cp.Uniform(0.39,0.405),
      "boundu(52)": cp.Uniform(8.00,12.00),
      "tftmp": cp.Uniform(4.0,5.5),}
mc_sampler = uq.sampling.MCSampler(vary=vary,n_mc_samples=75)

# Add mc_sampler to campaign
campaign.set_sampler(mc_sampler)

# Draw samples, execute and collate
campaign.execute(pool=client).collate(progress_bar=True)
samples = campaign.get_collation_result()
samples

# Specify the file path where you want to save the HDF file
hdf_file_path = 'uncertainties_data.h5'

# Save the DataFrame to HDF
samples.to_hdf(hdf_file_path, key='data', mode='w')



  0%|          | 0/700 [00:00<?, ?it/s]


 11%|█         | 76/700 [00:00<00:00, 759.90it/s]


 38%|███▊      | 263/700 [00:00<00:00, 1410.21it/s]


 63%|██████▎   | 442/700 [00:00<00:00, 1580.81it/s]


 88%|████████▊ | 613/700 [00:00<00:00, 1629.43it/s]


100%|██████████| 700/700 [00:00<00:00, 1535.30it/s]